# Milestone Project 2 - Blackjack Game
In this milestone project you will be creating a Complete BlackJack Card Game in Python.

Here are the requirements:

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

And most importantly:

* **You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!**


Feel free to expand this game. Try including multiple players. Try adding in Double-Down and card splits! Remember to you are free to use any resources you want and as always:

# HAVE FUN!

In [70]:
# https://tekeye.uk/playing_cards/svg-playing-cards

from IPython.display import display, clear_output
from ipywidgets import Button, Layout, BoundedIntText, HBox, VBox, GridBox, HTML
import random
import time


card_deck = {
    "clubs_2": 2,
    "clubs_3": 3,
    "clubs_4": 4,
    "clubs_5": 5,
    "clubs_6": 6,
    "clubs_7": 7,
    "clubs_8": 8,
    "clubs_9": 9,
    "clubs_10": 10,
    "clubs_ace": 11,
    "clubs_jack": 10,
    "clubs_king": 10,
    "clubs_queen": 10,
    "diamonds_2": 2,
    "diamonds_3": 3,
    "diamonds_4": 4,
    "diamonds_5": 5,
    "diamonds_6": 6,
    "diamonds_7": 7,
    "diamonds_8": 8,
    "diamonds_9": 9,
    "diamonds_10": 10,
    "diamonds_ace": 11,
    "diamonds_jack": 10,
    "diamonds_king": 10,
    "diamonds_queen": 10,
    "hearts_2": 2,
    "hearts_3": 3,
    "hearts_4": 4,
    "hearts_5": 5,
    "hearts_6": 6,
    "hearts_7": 7,
    "hearts_8": 8,
    "hearts_9": 9,
    "hearts_10": 10,
    "hearts_ace": 11,
    "hearts_jack": 10,
    "hearts_king": 10,
    "hearts_queen": 10,
    "spades_2": 2,
    "spades_3": 3,
    "spades_4": 4,
    "spades_5": 5,
    "spades_6": 6,
    "spades_7": 7,
    "spades_8": 8,
    "spades_9": 9,
    "spades_10": 10,
    "spades_ace": 11,
    "spades_jack": 10,
    "spades_king": 10,
    "spades_queen": 10
}

dealer = None
players = None
number_of_players = None
bets_input = None
dealer_title_html = '<p>Dealer</p>'
player_title_html = '<p>Player</p>'
active_player = None
actice_hand = None
    
class Hand:
    def __init__(self, card = None):
        self.cards = []
        if card is not None:
            self.cards.append(card)
        self.value = 0
        self.bet = 0
        self.blackjack = False
        self.finished_round = False
        self.bast = False
        
    def __lt__(self,other):
        return self.value < other.value
        
    def __gt__(self,other):
        return self.value > other.value
        
    def __eq__(self,other):
        return self.value == other.value
    
    def updateValue(self):
        value = 0
        number_of_aces = 0
        for card in self.cards:
            if "ace" in card:
                number_of_aces += 1
            value += card_deck[card]
        for i in range(number_of_aces):
            if value > 21:
                value -= 10
            else:
                break
        self.value = value
        return value 
    
    def addToHand(self, cards):
        self.cards += cards
        for card in cards:
            self.value += card_deck[card]
        if self.value > 21:
            value = 0
            number_of_aces = 0
            for card in self.cards:
                if "ace" in card:
                    number_of_aces += 1
                value += card_deck[card]
            for i in range(number_of_aces):
                if value > 21:
                    value -= 10
                else:
                    break
            self.value = value
        
    def display(self, end = None):
        card_width = 140
        cards_in_row = 4
        hand_html = f'<div style="display: grid; gap: 8px; grid-template-columns: repeat({cards_in_row},'\
        f'{card_width}px); justify-content: center;">'
        for card in self.cards[:end]:
            hand_html += '<div style="background-image:url(https://tekeye.uk/playing_cards/images/svg_playing_cards/'\
            f'fronts/{card}.svg); background-size: contain; background-repeat: no-repeat; height:200px;"></div>'
        if end is not None:
            hand_html += '<div style="background-image:url(https://tekeye.uk/playing_cards/images/svg_playing_cards/'\
            'backs/blue2.svg); background-size: contain; background-repeat: no-repeat; height:200px;"></div>' * \
            (len(self.cards) - end)
        hand_html += '</div>'
        return hand_html
        
    
class winLose:
    def lose(self, money):
        self.money -= money
        
    def win(self, money):
        self.money += money
        
    
class Dealer(winLose):
    def __init__(self, money = 1_000):
        self.hand = Hand()
        self.money = money
        self.deck = Deck()
    
    def clear_hand(self):
        self.hand = Hand()

        
class Player(winLose):
    def __init__(self, money = 100):
        self.hands = [Hand()]
        self.money = money
        self.had_split = False
        self.had_double_down = False
        
    def clear_hands(self):
        self.hands = [Hand()]
        
    
class Deck:
    number_of_decks = 6
    
    def __init__(self):
        self.deck = list(card_deck.keys()) * self.number_of_decks
        random.shuffle(self.deck)
    
    def shuffle(self):
        self.deck = list(card_deck.keys()) * self.number_of_decks
        random.shuffle(self.deck)
        
    def deal(self, cards = 1):
        to_deal = self.deck[-cards:]
        self.deck = self.deck[:-cards]
        return to_deal
    
    
def new_game_clicked(b):
    clear_output(wait=True)
    display(HBox([number_of_players_input, players_ok_button]))
    
        
def players_ok_button_clicked(b):
    global number_of_players
    number_of_players = number_of_players_input.value
    new_game()
    
    
def new_game():
    global dealer
    global players
    dealer = Dealer()
    players = [Player() for i in range(number_of_players)]
    betting()    
    

def betting():
    global players
    global bets_input
    clear_output(wait=True)
    if len(players) == 1:
        print("Place your bet:")
    else:
        print("Place your bets:")
    bets_input = []
    for player_number, player in enumerate(players, start=1):
        bets_input.append(
            BoundedIntText(
                value=10,
                min=1,
                max=player.money,
                step=1,
                disabled=False
            )
        )
        display(HBox([HTML(value=f"Player {player_number} ({player.money}$)"), bets_input[-1]]))
    display(betting_ok_button)
    
    
def betting_ok_button_clicked(b):
    global dealer
    global players
    global bets_input
    dealer.deck.shuffle()
    dealer.hand = Hand()
    dealer.hand.addToHand(dealer.deck.deal(2)) 
    if dealer.hand.value == 21:
         dealer.hand.blackjack = True
    for index, player in enumerate(players):
        player.hands = [Hand()]
        player.hands[0].bet = bets_input[index].value
        player.money -= player.hands[0].bet
        for i in range(2):
            player.hands[0].addToHand(dealer.deck.deal(1))
            clear_output(wait=True)
            print(f"Player {index+1}")
            display(HTML(player.hands[0].display()))
            time.sleep(0.8)
        if player.hands[0].value == 21:
            player.hands[0].blackjack = True
            player.hands[0].finished_round = True
            print("Blackjack!!!")
            time.sleep(0.8)
    game()
        
        
def game():
    global dealer
    global players
    global active_player
    global actice_hand
    
    for player_number, player in enumerate(players):
        for hand_number, hand in enumerate(player.hands):
            if not hand.finished_round:
                active_player = player_number
                actice_hand = hand_number
                if hand.value == 21:
                    game_screen(twenty_one=True)
                elif hand.value > 21:
                    game_screen(bust=True)
                else:
                    game_screen()
                break
    else:
        dealer_move()
    
def game_screen(twenty_one = False, bust = False):
    clear_output(wait=True)
    buttons = [new_game_button, hit_button, stand_button]
    if (not players[active_player].had_split or not players[active_player].had_double_down) and \
    len(players[active_player].hands[actice_hand].cards) == 2 and \
    players[active_player].money >= players[active_player].hands[actice_hand]:
        buttons += [double_button]
    if (not players[active_player].had_split or not players[active_player].had_double_down) and \
    len(players[active_player].hands[actice_hand].cards) == 2 and \
    card_deck[players[active_player].hands[actice_hand].cards[0]] == \
    card_deck[players[active_player].hands[actice_hand].cards[1]]:
        buttons += [split_button]
    if twenty_one:
        buttons = [twenty_one_button]
    elif bust:
        buttons = [bust_button]
    
    player_info = "Player "
    if len(players) != 1:
        player_info += f"{active_player+1} "
    if len(players[active_player].hands) != 1:
        player_info += f"Hand {actice_hand+1} "
    player_info += f"Score ({players[active_player].hands[actice_hand].value}) Money({players[active_player].money})"   
    
    display(
        GridBox(
            children=[
                HTML(value=dealer_title_html, layout=Layout(grid_area='dealer-title')),
                HTML(value=dealer.hand.display(1), layout=Layout(grid_area='dealer')),
                HTML(value=player_title_html, layout=Layout(grid_area='player-title')),
                HTML(value=players[active_player].hands[actice_hand].display(), layout=Layout(grid_area='player')),
                VBox(buttons, layout=Layout(grid_area='info'))
            ],
            layout=Layout(
                grid_template_columns='1fr 180px',
                grid_template_rows='40px 1fr 40px 1fr',
                grid_template_areas= '''
                    "dealer-title info"
                    "dealer info"
                    "player-title info"
                    "player info"
                    ''',
                grid_gap='8px')
            )
        )
        

    
def hit_button_clicked(b):
    global dealer
    global players
    global active_player
    global actice_hand
    
    players[active_player].hands[actice_hand].addToHand(dealer.deck.deal(1))
    game()


def stand_button_clicked(b):
    global players
    global active_player
    global actice_hand
    
    players[active_player].hands[actice_hand].finished_round = True
    game()


def split_button_clicked(b):
    global dealer
    global players
    global active_player
    global actice_hand
    
    players[active_player].had_split = True
    hand_to_split = players[active_player].hands.pop(actice_hand)
    players[active_player].hands += [Hand(hand_to_split.cards[0]), Hand(hand_to_split.cards[1])]
    players[active_player].hands[-1].addToHand(dealer.deck.deal(1))
    players[active_player].hands[-2].addToHand(dealer.deck.deal(1))
    game()


def double_button_clicked(b):
    global dealer
    global players
    global active_player
    global actice_hand
    
    players[active_player].had_double_down = True
    players[active_player].money -= players[active_player].hands[actice_hand].bet
    players[active_player].hands[actice_hand].bet += players[active_player].hands[actice_hand].bet
    players[active_player].hands[actice_hand].addToHand(dealer.deck.deal(1))
    game()
    

def dealer_move():
    dealer_move_screen()
    if not dealer.hand.blackjack:
        while dealer.hand.value < 17:
            time.sleep(0.8)
            dealer.hand.addToHand(dealer.deck.deal(1))
            dealer_move_screen()
    scoring()
            
    
def dealer_move_screen():
    clear_output(wait=True)
    print("Dealer move")
    display(HTML(value=dealer.hand.display()))
    print(f"Score ({dealer.hand.value})")
    
    
def scoring():
    if dealer.hand.
    if dealer.hand.blackjack:
        pass


def bust_button_clicked():
    players[player_number].hands[hand_number].finished_round = True
    game()

    
def twenty_one_button_clicked():
    players[player_number].hands[hand_number].finished_round = True
    game()

    
new_game_button = Button(description = "New game", layout=Layout(width='100px', height='28px'))
players_ok_button = Button(description = "Ok", layout=Layout(width='40px', height='28px'))
betting_ok_button = Button(description = "Ok", layout=Layout(width='40px', height='28px'))
number_of_players_input = BoundedIntText(value=1, min=1, max=7, step=1, description='Players:')
hit_button = Button(description = "Hit", layout=Layout(width='100px', height='28px'))
stand_button = Button(description = "Stand", layout=Layout(width='100px', height='28px'))
split_button = Button(description = "Split", layout=Layout(width='100px', height='28px'))
double_button = Button(description = "Double", layout=Layout(width='100px', height='28px'))
bust_button = Button(description = "Bust", layout=Layout(width='100px', height='28px'))
twenty_one_button = Button(description = "Twenty one", layout=Layout(width='100px', height='28px'))

new_game_button.on_click(new_game_clicked)
players_ok_button.on_click(players_ok_button_clicked)
betting_ok_button.on_click(betting_ok_button_clicked)
hit_button.on_click(hit_button_clicked)
stand_button.on_click(stand_button_clicked)
split_button.on_click(split_button_clicked)
double_button.on_click(double_button_clicked)
bust_button.on_click(bust_button_clicked)
twenty_one_button.on_click(twenty_one_button_clicked)

display(new_game_button)

Button(description='New game', layout=Layout(height='28px', width='100px'), style=ButtonStyle())

2
